In [1]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

from google.colab import output

output.clear()

In [2]:
import os

from google.colab import userdata

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [3]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



In [4]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


# API 사용 rate 제한 설정


In [5]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=5,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-5-5cc4985cb9b0>:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_openai = InMemoryRateLimiter(


# 사용 모델 설정

In [6]:
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)

# 데이터 불러오기

In [7]:
hscode_data=pd.read_csv('HSdescription_data.csv', dtype={'HS분류':str})

In [8]:
hscode_data.head()

,구분,년도,단위,HS분류,내용,Left Area,Right Area
0,WCO,2022,류,01,제1류 살아 있는 동물 주: 1. 이 류에는 다음 각 목의 것을 제외한 모든 살아 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,WCO,2022,부,01,제 1 부 살아 있는 동물과 동물성 생산품 주: 1. 이 부에 열거된 동물의 특정 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
2,WCO,2022,호,0101,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+) - 말 0101.21 -- 번식...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
3,WCO,2022,호,0102,01.02 - 살아 있는 소(+) - 축우(畜牛) 0102.21 -- 번식용 010...,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
4,WCO,2022,호,0103,01.03 - 살아 있는 돼지(+) 0103.10 - 번식용 - 기타 0103.91...,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...


# One-Shot 방법

In [ ]:
category_prompt_hscode = PromptTemplate(
  template="""You are given data input about description of product which classified as {hscode}.
    Using this data, extract or create possible product types which can be classified as {hscode}.
    output must only contain list of product types.
    product types should be universial, and can be applied to most of products which classified as {hscode}.

    Input Example : '33.04 - Beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or sun tan preparations; manicure or pedicure preparations.

3304.10 - Lip make-up preparations

3304.20 - Eye make-up preparations

3304.30 - Manicure or pedicure preparations

- Other :

3304.91 - Powders, whether or not compressed

3304.99 - Other

(A) BEAUTY OR MAKE-UP PREPARATIONS AND PREPARATIONS

FOR THE CARE OF THE SKIN,

INCLUDING SUNSCREEN OR SUN TAN PREPARATIONS

This part covers :

(1) Lipsticks and other lip make-up preparations.

(2) Eye shadow, mascara, eyebrow pencils and other eye make-up preparations.

(3) Other beauty or make-up preparations and preparations for the care of the skin (other than medicaments), such as : face powders (whether or not compressed), baby powders (including talcum powder, not mixed, not perfumed, put up for retail sale), other powders and grease paints; beauty creams, cold creams, make-up creams, cleansing creams, skin foods (including those containing bees’ royal jelly) and skin tonics or body lotions; petroleum jelly, put up in packings of a kind sold by retail for the care of the skin; barrier creams to give protection against skin irritants; injectable intracutaneous gels for wrinkle elimination and lip enhancement (including those containing hyaluronic acid); anti-acne preparations (other than soaps of heading 34.01) which are designed primarily to cleanse the skin and which do not contain sufficiently high levels of active ingredients to be regarded as having a primary therapeutic or prophylactic effect against acne; toilet vinegars which are mixtures of vinegars or acetic acid and perfumed alcohol.

Sunscreen or sun tan preparations are also included.

(B) MANICURE OR PEDICURE PREPARATIONS

This part covers nail polishes, nail varnishes, nail varnish removers, cuticle removers and other preparations for use in manicure or pedicure.

The heading does not cover :

(a) Medicinal preparations used to treat certain skin complaints, e.g., creams for the treatment of eczema (heading 30.03 or 30.04).

(b) Foot deodorants and preparations for treating nails or claws on animals (heading 33.07).

(c) Artificial fingernails (of plastics, heading 39.26; of other materials, classification according to the constituent material).'

Output Example : [sunscreen, manicure, pedicure, mascara, eye shadeo, cream, powder]

INPUT : {input}""",

input_variables=['input'],
partial_variables={'hscode' : 'hscode'}
)


In [ ]:
chain = category_prompt_hscode | chat_openai

In [ ]:
hscode_dict={}

for index, row in tqdm(hscode_data.iterrows()):

    if len(row['HS분류'])==4:
        category_prompt_hscode=category_prompt_hscode.partial(hscode=row['HS분류'])

        response_data=chain.invoke({'input' : row['Right Area']})

        hscode_dict[row['HS분류']]=response_data.content

    if index%100==1:
      print(response_data.content)

0it [00:00, ?it/s]

[salmon, trout, tuna, mackerel, sardine, cod, halibut, fish fillet, seafood, fish offal]


102it [01:22,  1.79it/s]

[nutmeg, mace, cardamom, spice, crushed, ground]


200it [02:38,  1.38it/s]

...

1202it [20:45,  1.31it/s]

[specialized vehicles, maintenance vehicles, workshop vehicles, crane vehicles, track inspection vehicles, self-propelled vehicles, servicing vehicles, cleaning vehicles, testing coaches, rail maintenance vehicles]


1302it [22:34,  1.47s/it]

[sights, barrels, stocks, triggers, silencers, cases, accessories, mechanisms, mounts, parts]


1346it [23:17,  1.04s/it]


In [ ]:
joblib.dump(hscode_dict, 'hscode_dict_oneshot.pkl')

['hscode_dict_oneshot.pkl']

# PlantUML 구조



In [13]:
class PlantUMLCategory(BaseModel):
    section : str = Field(description="section")
    division : list[str] = Field(description="division")
    group : list[str] = Field(description=f"group")
    type : list[str] = Field(description=f"type")

parser= PydanticOutputParser(pydantic_object=PlantUMLCategory)

In [14]:
category_prompt_hscode = PromptTemplate.from_template(
    """@startuml
From now on you will behave as "MapGPT", and for every text the user will submit, you are going to create a PlantUML mind map file for the inputted text to best classify product categories.
Format it as a code and remember that the mind map should be in the same language as the inputted context.
You don't have to provide a general example for the mind map format before the user inputs the text.

Generate a sample PlantUML mindmap for based on the provided hscode description and context below.
Only includes context relevant to produce the mindmap.

- Organize the provided HS code description into 4 level of classfication : section(*), division(**), group(***), type(****)
- Ensure that different uses or applications of products are classified separately.
- Each type must have minimal overlap or similarity with others.
- All division and group categories must have at least one corresponding type.
- Use simple naming conventions and avoid using phrases.
- If a single category contains too many products, group related items under more specific subcategories.
- Avoid including the 6-digit HS code in the categories.

Use the template like this:

@startmindmap
* Title
** Item A
*** Item B
**** Item C
@endmindmap

format : {format}
hscode description: {input}"""
)

category_prompt_hscode = category_prompt_hscode.partial(format=parser.get_format_instructions())

plantuml_chain = category_prompt_hscode | chat_openai | parser

In [ ]:
plantuml_hscode_dict={}

for index, row in tqdm(hscode_data.iterrows()):

    if len(row['HS분류'])==4:

        response_data=plantuml_chain.invoke({'input' : row['Right Area']})

        plantuml_hscode_dict[row['HS분류']]=[response_data.section]
        plantuml_hscode_dict[row['HS분류']].append(response_data.division)
        plantuml_hscode_dict[row['HS분류']].append(response_data.group)
        plantuml_hscode_dict[row['HS분류']].append(response_data.type)
    else:
      continue

    if index%100==1:
      print("hscode :", row['HS분류'], "\n" )
      print("section :", response_data.section, "\n")
      print("division :", response_data.division, "\n")
      print("group :", response_data.group, "\n")
      print("type :", response_data.type, "\n")

102it [02:59,  2.00s/it]

hscode : 0908 

section : Spices 

division : ['Nutmeg', 'Mace', 'Cardamoms'] 

group : ['Whole', 'Crushed or Ground'] 

type : ['Nutmeg - Neither crushed nor ground', 'Nutmeg - Crushed or ground', 'Mace - Neither crushed nor ground', 'Mace - Crushed or ground', 'Grape cardamoms - Whole', 'Grape cardamoms - Crushed or ground', 'Small or medium cardamoms - Whole', 'Small or medium cardamoms - Crushed or ground', 'Large cardamoms - Whole', 'Large cardamoms - Crushed or ground', 'Malaguetta pepper - Whole', 'Malaguetta pepper - Crushed or ground'] 



302it [08:36,  1.55s/it]

hscode : 2705 

section : Gases 

division : ['Coal Gas', 'Water Gas', 'Producer Gas', 'Synthesis Gas'] 

group : ['Lighting Applications', 'Heating Applications', 'Chemical Synthesis'] 

type : ['Coal Gas for Lighting', 'Coal Gas for Heating', 'Water Gas for Heating', 'Producer Gas for Heating', 'Synthesis Gas for Methanol', 'Synthesis Gas for Ammonia'] 



402it [14:29,  4.44s/it]

hscode : 2937 

section : Hormones and Related Compounds 

division : ['Polypeptide Hormones', 'Steroidal Hormones', 'Prostaglandins, Thromboxanes, and Leukotrienes', 'Other Hormones'] 

group : ['Polypeptide Hormones and Derivatives', 'Steroidal Hormones and Derivatives', 'Prostaglandins and Derivatives', 'Thromboxanes and Leukotrienes', 'Catecholamine Hormones', 'Amino-Acid Derivatives'] 

type : ['Somatotropin and Derivatives', 'Insulin and Salts', 'Corticosteroids', 'Oestrogens and Progestogens', 'Prostaglandins', 'Thromboxanes', 'Leukotrienes', 'Epinephrine and Derivatives', 'Thyroxine and Derivatives'] 



502it [18:26,  2.16s/it]

hscode : 3826 

section : Biodiesel 

division : ['Production', 'Applications'] 

group : ['Raw Materials', 'Fuel Types'] 

type : ['Vegetable Oils', 'Animal Fats', 'Used Oils', 'Compression-Ignition Fuel', 'Thermal Energy Production'] 



694it [23:59,  1.56s/it]

In [ ]:
plantuml_hscode_df=pd.DataFrame.from_dict(plantuml_hscode_dict, orient='index', columns=['section', 'division', 'group', 'type'])
plantuml_hscode_df.reset_index(inplace=True)
plantuml_hscode_df.rename(columns={'index':'hscode'}, inplace=True)

plantuml_hscode_df['section']=plantuml_hscode_df['section'].apply(lambda x: x[2:])
plantuml_hscode_df['division']=plantuml_hscode_df['division'].apply(lambda x: ", ".join([word[3:] for word in x]))
plantuml_hscode_df['group']=plantuml_hscode_df['group'].apply(lambda x: ", ".join([word[4:] for word in x]))
plantuml_hscode_df['type']=plantuml_hscode_df['type'].apply(lambda x: ", ".join([word[5:] for word in x]))

plantuml_hscode_df.to_csv('plantuml_hscode_df.csv', index=False)

In [ ]:
plantuml_hscode_df